#  AI Text Detection Project
This notebook implements the detection of AI-generated texts using embeddings and machine learning techniques. The steps include:

1. Loading the dataset.
2. Preprocessing the text data.
3. Using pre-computed embeddings to save time.
4. Training and evaluating classification models.


In [1]:
import pandas as pd

# Load the dataset
file_path = "R:/text_detection_project/datasets/Training_Essay_Data.csv"
data = pd.read_csv(file_path)

# Inspect the dataset
print(f"Dataset Shape: {data.shape}")
print(data.head())


Dataset Shape: (29145, 2)
                                                text  generated
0  Car-free cities have become a subject of incre...          1
1  Car Free Cities  Car-free cities, a concept ga...          1
2    A Sustainable Urban Future  Car-free cities ...          1
3    Pioneering Sustainable Urban Living  In an e...          1
4    The Path to Sustainable Urban Living  In an ...          1


In [2]:
import re

# Text Preprocessing
def preprocess_text(text):
    # Remove special characters, numbers, and punctuations
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower().strip()
    return text

# Apply preprocessing to the text column
data['cleaned_text'] = data['text'].apply(preprocess_text)

# Display a few samples of the cleaned text
print(data[['text', 'cleaned_text']].head())


                                                text  \
0  Car-free cities have become a subject of incre...   
1  Car Free Cities  Car-free cities, a concept ga...   
2    A Sustainable Urban Future  Car-free cities ...   
3    Pioneering Sustainable Urban Living  In an e...   
4    The Path to Sustainable Urban Living  In an ...   

                                        cleaned_text  
0  carfree cities have become a subject of increa...  
1  car free cities  carfree cities a concept gain...  
2  a sustainable urban future  carfree cities are...  
3  pioneering sustainable urban living  in an era...  
4  the path to sustainable urban living  in an ag...  


In [3]:
import numpy as np

# Path to the saved embeddings
embeddings_path = "R:/text_detection_project/datasets/bert_embeddings_combined.npy"

# Load the combined embeddings
embeddings = np.load(embeddings_path, allow_pickle=True)

# Check the shape of the embeddings
print(f"Shape of Embeddings: {embeddings.shape}")


Shape of Embeddings: (32256, 768)


In [4]:
# Ensure the dataset and embeddings have the same number of rows
if len(data) != len(embeddings):
    print(f"Mismatch detected: Dataset rows = {len(data)}, Embedding rows = {len(embeddings)}")
    data = data.iloc[:len(embeddings)]
    print(f"Dataset trimmed to match embeddings: {len(data)} rows.")

# Display the final dataset shape
print(f"Final Dataset Shape: {data.shape}")


Mismatch detected: Dataset rows = 29145, Embedding rows = 32256
Dataset trimmed to match embeddings: 29145 rows.
Final Dataset Shape: (29145, 3)


In [5]:
from sklearn.model_selection import train_test_split

# Features (embeddings) and labels
X = embeddings[:len(data)]  # Ensure embeddings match the trimmed dataset
y = data['generated'].values  # Assuming 'generated' column contains labels

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shape of the splits
print(f"Training set: {X_train.shape}, {y_train.shape}")
print(f"Testing set: {X_test.shape}, {y_test.shape}")


Training set: (23316, 768), (23316,)
Testing set: (5829, 768), (5829,)


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Initialize and train the Logistic Regression model
clf = LogisticRegression(max_iter=1000, random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.79      0.79      3539
           1       0.67      0.65      0.66      2290

    accuracy                           0.74      5829
   macro avg       0.72      0.72      0.72      5829
weighted avg       0.74      0.74      0.74      5829

Confusion Matrix:
[[2807  732]
 [ 802 1488]]


# Neural Network Implementation

In [7]:
import sklearn
print("Scikit-Learn version:", sklearn.__version__)


Scikit-Learn version: 1.6.0


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

# Define the Neural Network architecture
model = Sequential([
    Dense(512, activation='relu', input_shape=(768,)),  # First hidden layer
    Dropout(0.3),                                       # Dropout for regularization
    Dense(256, activation='relu'),                      # Second hidden layer
    Dropout(0.3),                                       # Dropout for regularization
    Dense(1, activation='sigmoid')                      # Output layer (binary classification)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Predict on the test set
y_pred_nn = (model.predict(X_test) > 0.5).astype("int32")

# Classification report and confusion matrix
print("Classification Report:")
print(classification_report(y_test, y_pred_nn))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_nn))


Epoch 1/10
583/583 [==============================] - 6s 9ms/step - loss: 0.5146 - accuracy: 0.7175 - val_loss: 0.4673 - val_accuracy: 0.7481
Epoch 2/10
583/583 [==============================] - 5s 8ms/step - loss: 0.4690 - accuracy: 0.7413 - val_loss: 0.4591 - val_accuracy: 0.7556
Epoch 3/10
583/583 [==============================] - 5s 9ms/step - loss: 0.4564 - accuracy: 0.7524 - val_loss: 0.4613 - val_accuracy: 0.7624
Epoch 4/10
583/583 [==============================] - 5s 8ms/step - loss: 0.4511 - accuracy: 0.7555 - val_loss: 0.4468 - val_accuracy: 0.7594
Epoch 5/10
583/583 [==============================] - 5s 9ms/step - loss: 0.4452 - accuracy: 0.7595 - val_loss: 0.4433 - val_accuracy: 0.7676
Epoch 6/10
583/583 [==============================] - 5s 9ms/step - loss: 0.4389 - accuracy: 0.7640 - val_loss: 0.4459 - val_accuracy: 0.7564
Epoch 7/10
583/583 [==============================] - 5s 9ms/step - loss: 0.4350 - accuracy: 0.7660 - val_loss: 0.4450 - val_accuracy: 0.7556
Epoch 

In [9]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Model checkpoint to save the best model
model_checkpoint = ModelCheckpoint('best_nn_model.h5', save_best_only=True, monitor='val_loss')

# Retrain the model with callbacks
history = model.fit(
    X_train, y_train,
    epochs=20,  # You can experiment with more epochs
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping, model_checkpoint]
)


Epoch 1/20
583/583 [==============================] - 5s 9ms/step - loss: 0.4276 - accuracy: 0.7662 - val_loss: 0.4386 - val_accuracy: 0.7633
Epoch 2/20
583/583 [==============================] - 5s 9ms/step - loss: 0.4277 - accuracy: 0.7679 - val_loss: 0.4389 - val_accuracy: 0.7599
Epoch 3/20
583/583 [==============================] - 5s 9ms/step - loss: 0.4243 - accuracy: 0.7724 - val_loss: 0.4424 - val_accuracy: 0.7669
Epoch 4/20
583/583 [==============================] - 5s 9ms/step - loss: 0.4205 - accuracy: 0.7733 - val_loss: 0.4428 - val_accuracy: 0.7590


In [10]:
from tensorflow.keras.models import load_model

# Load the best saved model
best_model = load_model('best_nn_model.h5')

# Evaluate the model on the test set
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Predict on the test set
y_pred_nn_refined = (best_model.predict(X_test) > 0.5).astype("int32")

# Classification report and confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

print("Classification Report:")
print(classification_report(y_test, y_pred_nn_refined))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_nn_refined))


183/183 [==============================] - 0s 2ms/step - loss: 0.4342 - accuracy: 0.7641
Test Accuracy: 0.76
183/183 [==============================] - 1s 3ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.89      0.82      3539
           1       0.77      0.57      0.66      2290

    accuracy                           0.76      5829
   macro avg       0.77      0.73      0.74      5829
weighted avg       0.76      0.76      0.76      5829

Confusion Matrix:
[[3141  398]
 [ 977 1313]]


In [13]:
import numpy as np
from sklearn.model_selection import train_test_split

# Load combined BERT embeddings
embedding_path = "R:/text_detection_project/datasets/bert_embeddings_combined.npy"
bert_embeddings_combined = np.load(embedding_path)

# Load the target labels
y = data['generated'].values  # Ensure this matches the dataset structure

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(bert_embeddings_combined, y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")


ValueError: Found input variables with inconsistent numbers of samples: [32256, 29145]

In [17]:
from sklearn.model_selection import train_test_split

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(bert_embeddings_combined, y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")


Training data shape: (23316, 768)
Testing data shape: (5829, 768)


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

# Define the Neural Network architecture
model = Sequential([
    Dense(512, activation='relu', input_shape=(768,)),  # First hidden layer
    Dropout(0.3),                                       # Dropout for regularization
    Dense(256, activation='relu'),                      # Second hidden layer
    Dropout(0.3),                                       # Dropout for regularization
    Dense(1, activation='sigmoid')                      # Output layer (binary classification)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Predict on the test set
y_pred_nn = (model.predict(X_test) > 0.5).astype("int32")

# Classification report and confusion matrix
print("Classification Report:")
print(classification_report(y_test, y_pred_nn))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_nn))


Epoch 1/20
583/583 [==============================] - 6s 9ms/step - loss: 0.5116 - accuracy: 0.7197 - val_loss: 0.4620 - val_accuracy: 0.7569
Epoch 2/20
583/583 [==============================] - 5s 8ms/step - loss: 0.4700 - accuracy: 0.7432 - val_loss: 0.4612 - val_accuracy: 0.7489
Epoch 3/20
583/583 [==============================] - 5s 9ms/step - loss: 0.4567 - accuracy: 0.7550 - val_loss: 0.4556 - val_accuracy: 0.7665
Epoch 4/20
583/583 [==============================] - 5s 9ms/step - loss: 0.4521 - accuracy: 0.7566 - val_loss: 0.4487 - val_accuracy: 0.7596
Epoch 5/20
583/583 [==============================] - 5s 9ms/step - loss: 0.4454 - accuracy: 0.7602 - val_loss: 0.4505 - val_accuracy: 0.7543
Epoch 6/20
583/583 [==============================] - 5s 9ms/step - loss: 0.4396 - accuracy: 0.7629 - val_loss: 0.4456 - val_accuracy: 0.7616
Epoch 7/20
583/583 [==============================] - 5s 9ms/step - loss: 0.4362 - accuracy: 0.7643 - val_loss: 0.4476 - val_accuracy: 0.7605
Epoch 

In [19]:
import numpy as np

# Function to preprocess and generate BERT embeddings for a single input
def predict_text(text, model, tokenizer, bert_model):
    # Preprocess text
    cleaned_text = clean_text(text)  # Reuse the clean_text function
    lemmatized_text = ' '.join(lemmatize_tokens(word_tokenize(cleaned_text)))

    # Generate BERT embeddings
    inputs = tokenizer(lemmatized_text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()

    # Reshape and predict
    prediction = model.predict(np.expand_dims(embedding, axis=0))
    predicted_class = "AI-generated" if prediction > 0.5 else "Human-written"
    
    return predicted_class, prediction[0][0]

# Sample texts to test
sample_texts = [
    "The sunset over the lake was breathtakingly beautiful.",
    "Utilizing advanced algorithms, this system optimizes resource allocation.",
    "Artificial intelligence is a rapidly evolving field.",
    "I love the way you describe the nuances of human emotion."
]

# Test each sample text
for text in sample_texts:
    predicted_class, confidence = predict_text(text, model, tokenizer, model)
    print(f"Text: {text}")
    print(f"Predicted Class: {predicted_class} (Confidence: {confidence:.2f})")
    print("-" * 50)


NameError: name 'tokenizer' is not defined

In [21]:
import numpy as np

# Function to preprocess and generate BERT embeddings for a single input
def predict_text(text, model, tokenizer, bert_model):
    # Preprocess text
    cleaned_text = clean_text(text)  # Reuse the clean_text function
    lemmatized_text = ' '.join(lemmatize_tokens(word_tokenize(cleaned_text)))

    # Generate BERT embeddings
    inputs = tokenizer(lemmatized_text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()

    # Reshape and predict
    prediction = model.predict(np.expand_dims(embedding, axis=0))
    predicted_class = "AI-generated" if prediction > 0.5 else "Human-written"
    
    return predicted_class, prediction[0][0]

# Sample texts to test
sample_texts = [
    "The sunset over the lake was breathtakingly beautiful.",
    "Utilizing advanced algorithms, this system optimizes resource allocation.",
    "Artificial intelligence is a rapidly evolving field.",
    "I love the way you describe the nuances of human emotion."
]

# Test each sample text
for text in sample_texts:
    predicted_class, confidence = predict_text(text, model, tokenizer, bert_model)
    print(f"Text: {text}")
    print(f"Predicted Class: {predicted_class} (Confidence: {confidence:.2f})")
    print("-" * 50)


NameError: name 'tokenizer' is not defined